In [7]:
!pip install langchain
!pip install openai
!pip install tiktoken
!pip install loguru
!pip install ansi2html

  Using cached langchain-0.0.247-py3-none-any.whl (1.4 MB)
  Using cached dataclasses_json-0.5.14-py3-none-any.whl (26 kB)
  Using cached langsmith-0.0.15-py3-none-any.whl (30 kB)
  Using cached openapi_schema_pydantic-1.2.4-py3-none-any.whl (90 kB)
  Using cached marshmallow-3.20.1-py3-none-any.whl (49 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl (4.7 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.9 MB/s eta 0:00:00


In [8]:
import os
os.environ["OPENAI_API_KEY"] = "key"
os.environ["SERPAPI_API_KEY"] = "key"
os.environ["SERPER_API_KEY"] = "key"

#1.callbacks

"LangChain" 提供了一个回调系统，可以让你在LLM应用的各个阶段进行操作，如日志记录，监控，流处理等。

使用API中的回调参数，你可以订阅这些事件。该参数是处理器对象的列表，它们应实现下面更详细描述的一个或多个方法。

回调处理器是实现了CallbackHandler接口的对象，每个可以订阅的事件都有一个方法。当事件被触发时，CallbackManager会在每个处理器上调用相应的方法。

LangChain 提供了几个内置处理器，如StdOutCallbackHandler，它将所有事件记录到 stdout。

在API中，大多数对象（链，模型，工具，代理等）都可以使用回调参数，可以在构造函数中定义，例如LLMChain(callbacks=[handler], tags=['a-tag'])，也可以在发出请求的run()/apply()方法中定义，例如chain.run(input, callbacks=[handler])。

详细地说，构造函数回调在非特定请求的日志记录，监控等用例中最有用，如你想记录到LLMChain的所有请求，你可以传递一个处理器给构造函数。请求回调在流处理等用例中最有用，如你想将单个请求的输出流传递到特定的websocket连接，你可以传递一个处理器给call()方法。

此外，当在对象的构造函数参数中设置 verbose 标志为 true，即使没有明确传递，StdOutCallbackHandler 也将被调用，这对于调试很有用，因为它会将所有事件记录到控制台。

In [ ]:
class BaseCallbackHandler:
    """可以用来处理langchain回调的基础回调处理器。"""

    def on_llm_start(
        self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any
    ) -> Any:
        """当LLM开始运行时执行。"""

    def on_chat_model_start(
        self, serialized: Dict[str, Any], messages: List[List[BaseMessage]], **kwargs: Any
    ) -> Any:
        """当聊天模型开始运行时执行。"""

    def on_llm_new_token(self, token: str, **kwargs: Any) -> Any:
        """新的LLM token生成时执行。只在启用流功能时可用。"""

    def on_llm_end(self, response: LLMResult, **kwargs: Any) -> Any:
        """当LLM运行结束时执行。"""

    def on_llm_error(
        self, error: Union[Exception, KeyboardInterrupt], **kwargs: Any
    ) -> Any:
        """当LLM出现错误时执行。"""

    def on_chain_start(
        self, serialized: Dict[str, Any], inputs: Dict[str, Any], **kwargs: Any
    ) -> Any:
        """当链开始运行时执行。"""

    def on_chain_end(self, outputs: Dict[str, Any], **kwargs: Any) -> Any:
        """当链结束运行时执行。"""

    def on_chain_error(
        self, error: Union[Exception, KeyboardInterrupt], **kwargs: Any
    ) -> Any:
        """当链出现错误时执行。"""

    def on_tool_start(
        self, serialized: Dict[str, Any], input_str: str, **kwargs: Any
    ) -> Any:
        """当工具开始运行时执行。"""

    def on_tool_end(self, output: str, **kwargs: Any) -> Any:
        """当工具结束运行时执行。"""

    def on_tool_error(
        self, error: Union[Exception, KeyboardInterrupt], **kwargs: Any
    ) -> Any:
        """当工具出现错误时执行。"""

    def on_text(self, text: str, **kwargs: Any) -> Any:
        """处理任意文本。"""

    def on_agent_action(self, action: AgentAction, **kwargs: Any) -> Any:
        """当agent采取行动时执行。"""

    def on_agent_finish(self, finish: AgentFinish, **kwargs: Any) -> Any:
        """当agent结束时执行。"""


In [9]:
from langchain.callbacks import StdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

handler = StdOutCallbackHandler()
llm = OpenAI()
prompt = PromptTemplate.from_template("1 + {number} = ")

#初始化的时候定义回调
#构造函数回调：首先，我们在初始化我们的链时显式设置
chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])
chain.run(number=2)

#通过设置verbose标志，隐性调用回调
#使用verbose标志：然后，我们使用 verbose 标志来实现相同的结果
chain = LLMChain(llm=llm, prompt=prompt, verbose=True)
chain.run(number=2)

#在请求的时候直接设置回调
#我们使用请求的 callbacks 来实现相同的结果
chain = LLMChain(llm=llm, prompt=prompt)
chain.run(number=2, callbacks=[handler])



> Entering new LLMChain chain...
Prompt after formatting:
1 + 2 = 

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
1 + 2 = 

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
1 + 2 = 

> Finished chain.


'\n3'

#2.Async callbacks
通过重写基础类中的函数，实现基本的客制化要求

如果你计划使用异步API，推荐使用AsyncCallbackHandler来避免阻塞运行循环。尽管在使用异步方法运行你的llm/chain/tool/agent时，你使用同步的CallbackHandler也能工作，但实际上，它将会通过run_in_executor被调用，如果你的CallbackHandler不是线程安全的，这可能会引发问题。

示例中，定义了两个回调处理器类，一个是同步处理器MyCustomSyncHandler，另一个是异步处理器MyCustomAsyncHandler。在异步处理器中，当链开始运行和结束运行时，都会打印出一段消息，并在其中插入了一个异步的延时。

In [11]:
import asyncio
from typing import Any, Dict, List

from langchain.chat_models import ChatOpenAI
from langchain.schema import LLMResult, HumanMessage
from langchain.callbacks.base import AsyncCallbackHandler, BaseCallbackHandler


class MyCustomSyncHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        print(f"同步处理器：token: {token}")

    def on_llm_start(
        self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any
    ) -> None:
        """当链开始运行时运行。"""

        print("同步处理：LLM正在开始")

    def on_llm_end(self, response: LLMResult, **kwargs: Any) -> None:
        """当链结束运行时运行。"""

        print("同步处理：LLM正在结束")


class MyCustomAsyncHandler(AsyncCallbackHandler):
    """可以用来处理来自langchain的回调的异步回调处理器。"""

    def on_llm_new_token(self, token: str, **kwargs) -> None:
        print(f"异步处理器：token: {token}")


    async def on_llm_start(
        self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any
    ) -> None:
        """当链开始运行时运行。"""
        print("zzzz....")
        #await asyncio.sleep(0.3)
        #class_name = serialized["name"]
        print("异步处理：LLM正在开始")

    async def on_llm_end(self, response: LLMResult, **kwargs: Any) -> None:
        """当链结束运行时运行。"""
        print("zzzz....")
        #await asyncio.sleep(0.3)
        print("异步处理：LLM正在结束")


# 为了启用流式处理，我们在ChatModel构造函数中传入`streaming=True`
# 在列表中传入自定义处理器
chat = ChatOpenAI(
    max_tokens=25,
    streaming=True,
    callbacks=[MyCustomAsyncHandler()],
)

await chat.agenerate([[HumanMessage(content="告诉我一个笑话")]])


zzzz....
异步处理：LLM正在开始
异步处理器：token: 
异步处理器：token: 当
异步处理器：token: 老
异步处理器：token: 师
异步处理器：token: 问
异步处理器：token: 小
异步处理器：token: 明
异步处理器：token: ：“
异步处理器：token: 小
异步处理器：token: 明
异步处理器：token: ，
异步处理器：token: 你
异步处理器：token: 知
异步处理器：token: 道
异步处理器：token: 什
异步处理器：token: 么
异步处理器：token: 是
异步处理器：token: 心
异步处理器：token: 理
异步处理器：token: 学
异步处理器：token: 吗
异步处理器：token: ？”
异步处理器：token: 小
异步处理器：token: 
zzzz....
异步处理：LLM正在结束


LLMResult(generations=[[ChatGenerationChunk(text='当老师问小明：“小明，你知道什么是心理学吗？”小', generation_info=None, message=AIMessageChunk(content='当老师问小明：“小明，你知道什么是心理学吗？”小', additional_kwargs={}, example=False))]], llm_output={'token_usage': {}, 'model_name': 'gpt-3.5-turbo'}, run=[RunInfo(run_id=UUID('5fa0e70d-7039-40b0-860c-f8258d76b01d'))])

#3.logging file

使用 FileCallbackHandler，该处理器与 StdOutCallbackHandler 功能相同，但将输出写入文件而非标准输出。示例还使用了 loguru 库来记录未被处理器捕获的其他输出。

In [12]:
!pip install loguru

In [13]:
from loguru import logger
from langchain.callbacks import FileCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

logfile = "output.log"

logger.add(logfile, colorize=True, enqueue=True)
handler = FileCallbackHandler(logfile)

llm = OpenAI()
prompt = PromptTemplate.from_template("1 + {number} = ")


chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler], verbose=True)
answer = chain.run(number=2)
logger.info(answer)



> Entering new LLMChain chain...
Prompt after formatting:
1 + 2 = 


2023-07-31 12:42:22.268 | INFO     | __main__:<cell line: 18>:18 - 

3



> Finished chain.


In [14]:
!pip install ansi2html

Ansi2HTMLConverter是一个类，它的作用是将ANSI编码的文本转换为HTML。ANSI编码通常用于控制台输出，例如在终端中显示的彩色文本或带格式的文本（如粗体、斜体等）。这种编码方式可以用特殊的代码来表示不同的颜色和格式，但这些代码在普通的文本编辑器或网页中通常无法正确显示。

Ansi2HTMLConverter的convert方法将含有这些特殊代码的文本转换为可以在网页中正确显示的HTML。在这段代码中，convert方法将从日志文件读取的内容（content）转换为HTML，并将结果赋值给html变量。

In [15]:
from IPython.display import display, HTML
from ansi2html import Ansi2HTMLConverter

with open("output.log", "r") as f:
    content = f.read()

conv = Ansi2HTMLConverter()
html = conv.convert(content, full=True)

display(HTML(html))

#4.Multiple callback handlers
这段代码定义了两个自定义的回调处理器类：MyCustomHandlerOne和MyCustomHandlerTwo，这两个类都继承自BaseCallbackHandler，并且重写了其部分方法，以便于在特定的事件发生时执行自定义的操作。

In [16]:
from typing import Dict, Union, Any, List

from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema import AgentAction
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.callbacks import tracing_enabled
from langchain.llms import OpenAI


# 首先，定义自定义回调处理器实现
class MyCustomHandlerOne(BaseCallbackHandler):
    def on_llm_start(
        self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any
    ) -> Any:
        # 当语言模型开始运行时调用
        class_name = serialized.get("name", serialized.get("id", ["<unknown>"])[-1])
        print(f"MyCustomHandlerOne：on_llm_start "+class_name)
        #print(f"on_llm_start {serialized['name']}")

    def on_llm_new_token(self, token: str, **kwargs: Any) -> Any:
        # 当语言模型生成新的 token 时调用
        print(f"MyCustomHandlerOne：on_new_token {token}")

    def on_llm_error(
        self, error: Union[Exception, KeyboardInterrupt], **kwargs: Any
    ) -> Any:
        # 当语言模型出现错误时运行
        pass

    def on_chain_start(
        self, serialized: Dict[str, Any], inputs: Dict[str, Any], **kwargs: Any
    ) -> Any:
        # 当链开始运行时调用
        class_name = serialized.get("name", serialized.get("id", ["<unknown>"])[-1])
        print(f"MyCustomHandlerOne：on_llm_start "+class_name)
        #print(f"on_chain_start {serialized['name']}")

    def on_tool_start(
        self, serialized: Dict[str, Any], input_str: str, **kwargs: Any
    ) -> Any:
        # 当工具开始运行时调用
        class_name = serialized.get("name", serialized.get("id", ["<unknown>"])[-1])
        print(f"MyCustomHandlerOne：on_llm_start "+class_name)
        #print(f"on_tool_start {serialized['name']}")

    def on_agent_action(self, action: AgentAction, **kwargs: Any) -> Any:
        # 当 agent 执行动作时调用
        print(f"MyCustomHandlerOne：on_agent_action {action}")


class MyCustomHandlerTwo(BaseCallbackHandler):
    def on_llm_start(
        self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any
    ) -> Any:
        # 当语言模型开始运行时调用
        class_name = serialized.get("name", serialized.get("id", ["<unknown>"])[-1])
        print(f"MyCustomHandlerTwo：on_llm_start "+class_name)
        #print(f"on_llm_start (我是第二个处理器!!) {serialized['name']}")


# 实例化处理器
handler1 = MyCustomHandlerOne()
handler2 = MyCustomHandlerTwo()

# 设置 agent。只有 `llm` 会为 handler2 发出回调
#handler2 负责创建OpenAI LLM的回调工作
llm = OpenAI(temperature=0, streaming=True, callbacks=[handler2])
tools = load_tools(["llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION)

# handler1在 Agent 执行中涉及的每个对象（llm，llmchain，tool，agent 执行器）都将为 handler1 发出回调
agent.run("2的0.235次幂是多少?", callbacks=[handler1])


MyCustomHandlerOne：on_llm_start AgentExecutor
MyCustomHandlerOne：on_llm_start LLMChain
MyCustomHandlerOne：on_llm_start OpenAI
MyCustomHandlerTwo：on_llm_start OpenAI
MyCustomHandlerOne：on_new_token  I
MyCustomHandlerOne：on_new_token  need
MyCustomHandlerOne：on_new_token  to
MyCustomHandlerOne：on_new_token  calculate
MyCustomHandlerOne：on_new_token  this
MyCustomHandlerOne：on_new_token  exponential
MyCustomHandlerOne：on_new_token  value
MyCustomHandlerOne：on_new_token .
MyCustomHandlerOne：on_new_token 
Action
MyCustomHandlerOne：on_new_token :
MyCustomHandlerOne：on_new_token  Calculator
MyCustomHandlerOne：on_new_token 
Action
MyCustomHandlerOne：on_new_token  Input
MyCustomHandlerOne：on_new_token :
MyCustomHandlerOne：on_new_token  2
MyCustomHandlerOne：on_new_token ^
MyCustomHandlerOne：on_new_token 0
MyCustomHandlerOne：on_new_token .
MyCustomHandlerOne：on_new_token 235
MyCustomHandlerOne：on_new_token 
MyCustomHandlerOne：on_agent_action AgentAction(tool='Calculator', tool_input='2^0.235', lo

'1.1769067372187674'

#5.Token Counter
这段代码使用了 langchain 这个 Python 库来创建和使用一个 OpenAI 对象，该对象将被用来生成文本。它主要演示了如何使用 get_openai_callback() 作为一个上下文管理器来追踪生成的文本的 token 总数，并进行了一些断言测试来检验生成的 token 总数是否符合预期。

In [17]:
import asyncio

from langchain.callbacks import get_openai_callback
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)
with get_openai_callback() as cb:
    output= llm("What is the square root of 4?")

total_tokens = cb.total_tokens
print(cb.total_tokens)
print(output)
assert total_tokens > 0

with get_openai_callback() as cb:
    llm("What is the square root of 4?")
    llm("What is the square root of 4?")
print(cb.total_tokens)
assert cb.total_tokens == total_tokens * 2

# You can kick off concurrent runs from within the context manager
with get_openai_callback() as cb:
    await asyncio.gather(
        *[llm.agenerate(["What is the square root of 4?"]) for _ in range(3)]
    )
print(cb.total_tokens)
assert cb.total_tokens == total_tokens * 3

# The context manager is concurrency safe
task = asyncio.create_task(llm.agenerate(["What is the square root of 4?"]))
with get_openai_callback() as cb:
    await llm.agenerate(["What is the square root of 4?"])

await task
print(cb.total_tokens)
assert cb.total_tokens == total_tokens

18


The square root of 4 is 2.
36
54
18
